In [2]:
import requests
from bs4 import BeautifulSoup
import pandas as pd
import os
import time
import random

# 티커 데이터를 수집하고 저장하는 클래스 정의
class TickerDataCollector:
    def __init__(self, tickers_file, financial_results_file):
        # 클래스 초기화: 티커 파일 경로와 재무 결과 파일 경로 설정
        self.tickers_file = tickers_file
        self.financial_results_file = financial_results_file
        self.tickers = self.load_tickers()  # 티커 목록 로드
        self.results_df = self.load_results()  # 이전에 저장된 재무 데이터 로드

    # 티커 목록을 로드하는 메서드
    def load_tickers(self):
        if os.path.exists(self.tickers_file):  # 파일 존재 여부 확인
            df = pd.read_csv(self.tickers_file)  # CSV 파일 읽기
            tickers = df["Tickers"].tolist()  # 'Tickers' 열을 리스트로 변환
            tickers = [str(x) for x in tickers]  # 모든 티커를 문자열로 변환
            tickers.sort()  # 티커 정렬
            return tickers  # 정렬된 티커 목록 반환
        else:
            # 파일이 존재하지 않을 경우 오류 발생
            raise FileNotFoundError(f"{self.tickers_file} not found.")

    # 이전에 저장된 재무 데이터를 로드하는 메서드
    def load_results(self):
        if os.path.exists(self.financial_results_file):  # 파일 존재 여부 확인
            return pd.read_csv(self.financial_results_file, index_col=0)  # 기존 데이터프레임 로드
        else:
            # 파일이 없으면 새로운 데이터프레임 생성
            columns = [
                'Index', 'Market Cap', 'Income', 'Sales', 'Book/sh', 'Cash/sh', 
                'Dividend Est.', 'Dividend TTM', 'Dividend Ex-Date', 'Employees', 
                'Option/Short', 'Sales Surprise', 'SMA20', 'P/E', 'Forward P/E', 
                'PEG', 'P/S', 'P/B', 'P/C', 'P/FCF', 'Quick Ratio', 'Current Ratio', 
                'Debt/Eq', 'LT Debt/Eq', 'EPS Surprise', 'SMA50', 'EPS (ttm)', 
                'EPS next Y', 'EPS next Q', 'EPS this Y', 'EPS next 5Y', 'EPS past 5Y', 
                'Sales past 5Y', 'EPS Y/Y TTM', 'Sales Y/Y TTM', 'EPS Q/Q', 'Sales Q/Q', 
                'SMA200', 'Insider Own', 'Insider Trans', 'Inst Own', 'Inst Trans', 
                'ROA', 'ROE', 'ROI', 'Gross Margin', 'Oper. Margin', 'Profit Margin', 
                'Payout', 'Earnings', 'Trades', 'Shs Outstand', 'Shs Float', 
                'Short Float', 'Short Ratio', 'Short Interest', '52W Range', 
                '52W High', '52W Low', 'RSI (14)', 'Recom', 'Rel Volume', 
                'Avg Volume', 'Volume', 'Perf Week', 'Perf Month', 'Perf Quarter', 
                'Perf Half Y', 'Perf Year', 'Perf YTD', 'Beta', 'ATR (14)', 
                'Volatility', 'Target Price', 'Prev Close', 'Price', 'Change'
            ]
            return pd.DataFrame(columns=columns)  # 빈 데이터프레임 반환

    # 웹 페이지에서 데이터를 가져오는 메서드
    def fetch_data(self, ticker):
        url = f'https://finviz.com/quote.ashx?t={ticker}&p=d'  # 요청할 URL 설정
        headers = {
            'User-Agent': 'Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/114.0.0.0 Safari/537.36',
            'Referer': 'https://example.com',
            'Accept-Language': 'ko-KR,ko;q=0.9',
            'Accept': 'text/html,application/xhtml+xml,application/xml;q=0.9,*/*;q=0.8',
            'Connection': 'keep-alive'
        }
        
        response = requests.get(url, headers=headers)  # 웹 페이지 요청
        while response.status_code == 429:  # 너무 많은 요청으로 차단된 경우
            print("너무 많은 요청.. 재시도 중")
            time.sleep(random.uniform(3, 5))  # 재시도 전 3~5초 대기
            response = requests.get(url, headers=headers)  # 다시 요청
        return BeautifulSoup(response.text, "html.parser")  # HTML 파싱하여 반환

    # HTML 테이블을 파싱하여 데이터 리스트로 변환하는 메서드
    def parse_table(self, soup):
        table = soup.find("table", {"class": "snapshot-table2"})  # 테이블 찾기
        rows = table.find_all("tr")  # 모든 행 가져오기
        table_data = []

        for row in rows:
            cells = row.find_all("td")  # 각 행의 모든 셀 가져오기
            row_data = [cell.get_text(strip=True) for cell in cells]  # 텍스트 추출 및 공백 제거
            table_data.append(row_data)  # 추출한 데이터 리스트에 추가
        
        # 데이터프레임으로 변환
        df = pd.DataFrame(table_data).reset_index(drop=True)
        value_list = []
        for i in range(0, 12, 2):  # 필요한 데이터만 추출
            value_list.extend(df[i + 1].tolist())  # 짝수 열의 데이터만 추가
        return value_list  # 최종 데이터 리스트 반환

    # 모든 티커에 대해 데이터를 수집하는 메서드
    def collect_data(self):
        start_ticker_index = len(self.results_df)  # 이미 수집된 데이터 개수
        start_ticker = self.tickers[start_ticker_index]  # 시작할 티커

        for ticker in self.tickers[start_ticker_index:]:
            print(f"현재 탐색 티커: {ticker}")
            soup = self.fetch_data(ticker)  # 웹 페이지 데이터 가져오기
            value_list = self.parse_table(soup)  # 테이블 데이터 파싱
            self.results_df.loc[ticker] = value_list  # 데이터프레임에 추가
            self.save_results()  # 결과 저장

    # 수집된 데이터를 CSV 파일에 저장하는 메서드
    def save_results(self):
        self.results_df.to_csv(self.financial_results_file, index=True)  # CSV 파일로 저장

# 프로그램 실행 부분
if __name__ == "__main__":
    tickers_file = 'tickers.csv'  # 티커 파일 경로
    financial_results_file = 'financial_results.csv'  # 결과 파일 경로
    print("test")

    collector = TickerDataCollector(tickers_file, financial_results_file)  # 객체 생성
    collector.collect_data()  # 데이터 수집 시작


test
현재 탐색 티커: CDXC
현재 탐색 티커: CDXS
현재 탐색 티커: CDZI
현재 탐색 티커: CE
현재 탐색 티커: CEAD
현재 탐색 티커: CECO
현재 탐색 티커: CEF
현재 탐색 티커: CEG
현재 탐색 티커: CEIX
현재 탐색 티커: CELC
현재 탐색 티커: CELH
현재 탐색 티커: CELU
현재 탐색 티커: CELZ


KeyboardInterrupt: 